In [1]:
import os
import torch
import pytorch_lightning as L
import matplotlib.pyplot as plt
from omegaconf import DictConfig

%matplotlib inline
plt.rcParams['image.interpolation'] = 'nearest'

%load_ext autoreload
%autoreload 2

print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory)

# Get number of cores
print(os.cpu_count())

# Get number of threads
print(torch.get_num_threads())

In [2]:
from pytorch_lightning.loggers import WandbLogger

from src import RoadDataModule, RoadModel, LogPredictionsCallback, val_checkpoint, regular_checkpoint


def main(cfg: DictConfig) -> None:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = RoadModel(cfg, device)
    datamodule = RoadDataModule(cfg)

    wandb_logger = WandbLogger(project="road-segmentation", name="test")

    trainer = L.Trainer(max_epochs=5, accelerator="gpu", devices=1, logger=wandb_logger, callbacks=[
        LogPredictionsCallback(),
        val_checkpoint,
        regular_checkpoint
    ])
    # trainer.fit(model, datamodule=datamodule)
    trainer.test(model, datamodule=datamodule)
    # trainer.predict(model, datamodule=datamodule)

In [3]:
from hydra import compose, initialize

with initialize(version_base=None, config_path="conf"):
    config = compose(config_name="config")
    main(config)